## Find Guider mode ROIs for ComCam
Craig Lage - 02Dec-24

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import astropy.io.fits as pf
from lsst.daf.butler import Butler
import lsst.summit.utils.butlerUtils as butlerUtils
from lsst.summit.utils.plotting import plot
from astroquery.gaia import Gaia
from astropy.coordinates import SkyCoord
import astropy.units as u

In [ ]:
butler = butlerUtils.makeDefaultButler("LSSTComCam")
instrument = 'LSSTComCam'

# Find the ROIs.  This cell takes longer but finds the magnitudes.

In [ ]:
expId = 2024120100406

detNames = ['S00', 'S01', 'S21', 'S22']
for i, detector in enumerate([0, 1, 7, 8]):
    exp = butler.get('calexp', detector=detector, visit=expId, instrument=instrument)
    cat = butler.get('src', detector=detector, visit=expId, instrument=instrument)
    sources = cat.asAstropy()
    sources.keep_columns(['base_SdssCentroid_x', 'base_SdssCentroid_y', \
                          'coord_ra', 'coord_dec', 'base_PsfFlux_chi2'])
    sources.sort('base_PsfFlux_chi2', reverse=True)
    
    index = 0
    while True:
        x = sources[index]['base_SdssCentroid_x']
        y = sources[index]['base_SdssCentroid_y']
        for amp in exp.getDetector().getAmplifiers():
            name = amp.getName()
            bbox = amp.getBBox()
            if x > bbox.beginX and x < bbox.endX and y > bbox.beginY and y < bbox.endY:
                ampX = x - bbox.beginX
                ampY = y - bbox.beginY
                start_row = int(ampX - 50)
                start_col = int(ampY - 50)
                break
        # Check that there are no stars nearby                
        coord = SkyCoord(sources[index]['coord_ra'], sources[index]['coord_dec'], unit="rad")
        result = Gaia.query_object_async(coordinate=coord, radius = 5.0 * u.arcsec)
        print(f" GAIA found {len(result)} stars")

        if start_row > 50 and start_col > 50 and start_row < 450 and \
        start_col < 1900 and len(result) == 1:
            break
        else:
            index += 1

    print(f"For {detNames[i]}, segment={name}, start_row={start_row}, start_col={start_col}")
    # Extract magnitudes from the result
    g_mag = result['phot_g_mean_mag']
    bp_mag = result['phot_bp_mean_mag']
    rp_mag = result['phot_rp_mean_mag']

    # Loop over each row and print G, BP, and RP magnitudes on the same line
    print("g_mag, bp_mag, rp_mag")
    for g, bp, rp in zip(g_mag, bp_mag, rp_mag):
        print(f"{g:.2f}, {bp:.2f}, {rp:.2f}")

    x = plot(exp, stretch='asinh')
    ax = x.get_axes()[0]
    ax.scatter([sources[index]['base_SdssCentroid_x']], [sources[index]['base_SdssCentroid_y']], \
               marker='x', s=500, color='green')
        
    plt.savefig(f"/home/c/cslage/u/ComCam/images/Guider_{expId}_{detector}.png")
